In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
import random

import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

C:\Users\sgold\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
learning_rate = 0.001
batch_size = 10
epochs = 10

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)

In [3]:
train_path = 'C:/Users/sgold/laba5/train'
test_path = 'C:/Users/sgold/laba5/test'
val_path = 'C:/Users/sgold/laba5/val'
path_to_dataset=r"C:/Users/sgold/laba5/dataset/"

df = pd.read_csv(os.path.join(path_to_dataset, "C:/Users/sgold/laba5/dataset_csv.csv"), header=None, delimiter=";", names=['path_to_image', 'label'])
if not os.path.isdir(train_path):
    os.mkdir(train_path)

if not os.path.isdir(test_path):
    os.mkdir(test_path)

if not os.path.isdir(val_path):
    os.mkdir(val_path)

def train_loading(df, path, i):
    image_path = df.path_to_image[i]
    image = cv2.imread(image_path)
    cv2.imwrite(os.path.join(path, f'{df.label[i]}.{i}.jpg'), image)

def test_loading(df, path, i):
    image_path = df.path_to_image[i]
    image = cv2.imread(image_path)
    if i - 880 > 109: 
        i = i - 1870
    else:
        i = i - 880
    cv2.imwrite(os.path.join(path, f'{i}.jpg'), image)

for i in range(880):
    train_loading(df, train_path, i)

for i in range(880, 990):
    train_loading(df, test_path, i)

for i in range(990, 1100):
    train_loading(df, val_path, i)

for i in range(1100, 1980):
    train_loading(df, train_path, i)

for i in range(1980, 2090):
    train_loading(df, test_path, i)

for i in range(2090, 2200):
    train_loading(df, val_path, i)
    

train_list = glob.glob(os.path.join(train_path,'*.jpg'))
test_list = glob.glob(os.path.join(test_path, '*.jpg'))
train_list, val_list = train_test_split(train_list, test_size=0.1)
    